### Set Parent directory

In [1]:
#  set parent directory to import from
import sys
sys.path.append('../../')

In [2]:
from models.architecture.rnn_net import RnnPredictCandles
from tools.dataset import Dataset
from models.processing.create_features import createFeatures
from models.processing.processing_data import Transformer

import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [50]:
dataset = Dataset().get_data(days=7, ticker='BTCUSDT', ts='5m')

In [51]:
features = createFeatures().create_features(dataset[['open', 'high', 'low', 'close']])
cols = features.columns
transformer = Transformer()
X, y, indexes = transformer.transform_to_lstm_input(
    data=features,
    cols=cols,
    window_size=12,
    step_size=0,
    ahead=12
)

0it [00:00, ?it/s]

[X] Ooops!!! It's True, the series 'volume' was not found in open, high, low, close


/home/serg/projects/onramp/quant-ai/models/processing/create_features.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

39it [00:00, 68.47it/s]


In [52]:
X_split = transformer.split_data(X, 0.1)
y_split = transformer.split_data(y, 0.1)
indexes = transformer.split_data(indexes, 0.1)

model = RnnPredictCandles(
    input_shape=X.shape[1:],
    output_shape=y.shape[1:],
    epochs=500
)
model.verbose = 0
model.batch_size = 32
model.build_model()
model.fit_train_test(X_split[0], y_split[0], X_split[1], y_split[1])


In [53]:
predicted = model.predict(X_split[1])
predicted = transformer.inverse_min_max_scaler_y(predicted[-1])
print(f'predicted: {predicted}')

57/57 [==============================] - 1s 3ms/step
predicted: [[20270.49  20284.688 20210.348 20231.139]
 [20222.156 20254.143 20186.494 20188.756]
 [20191.324 20228.53  20160.258 20181.068]
 [20184.248 20184.924 20118.887 20135.219]
 [20132.586 20153.195 20080.104 20109.764]
 [20094.664 20131.188 20054.72  20080.188]
 [20093.77  20103.963 20036.35  20045.148]
 [20060.406 20089.523 20014.63  20043.086]
 [20042.564 20063.93  19991.537 20029.275]
 [20015.54  20037.604 19993.295 19997.828]
 [19999.129 20019.566 19970.021 19987.84 ]
 [19994.1   20015.936 19943.045 19966.96 ]]


# Predict real time for the next 12 Hours (candles)

In [54]:
expected_prices = pd.DataFrame(predicted, columns=['open', 'high', 'low', 'close'])
fig = go.Figure(data=[go.Candlestick(
                x=expected_prices.index,
                open=expected_prices['open'],
                high=expected_prices['high'],
                low=expected_prices['low'],
                close=expected_prices['close'])])
fig.show()